In [149]:
from pyspark.sql import SparkSession

#### we put our files in the cloned git so git can track it as we make changes

In [147]:
import pandas as pd

In [148]:
#app name is just the name of the app
spark = SparkSession.builder.appName('rheeza').getOrCreate() #connection # creating an instance of connection

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
 # can take multiline attribute, to remove an added column called corrupt

In [ ]:
trials_df = spark.read.json('dataset.json', multiLine=True)

## Understanding the dataset

In [ ]:
trials_df.show(4)


In [ ]:
#pd.read_json('dataset.json') # Just trying to observe the difference here spark vs pandas read json

In [ ]:
# To see the structure of the dataset, datatypes struct is nested, long is an integer type
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [ ]:
trials_df.show(3)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
only showing top 3 r

In [ ]:
trials_df.dtypes # data types in list of tuples

In [ ]:
trials_df.columns # getting the name of columns

In [ ]:
from pyspark.sql.functions import isnull, count, isnan, when, col, date_format, to_date

In [ ]:
# Let's flatten our json file so each attribute appears as a column
# flatten is making all each attribute in json appear on an individual column--no nesting, e.g branch will not be under clinician

In [ ]:
# Lazy way of doing this, split the columns
columns = [
 'ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant'
]



### Checking null columns

In [ ]:
trials_df.select(columns).show() # This is just seeing the nested part - flattening

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [ ]:
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [152]:
# importing functions to count null value
# import all the functions module of pyspark

from pyspark.sql import functions as fn

In [153]:
# Or simply import the needed functions
from pyspark.sql.functions import isnull, count, isnan, when, col, date_format, to_date

In [ ]:
# To count all columns with null values, also create a condition to sieve --all columns
# when is null pick column

In [154]:
# trials_df.select([ fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]) # perform fn when isnull =
# null_columns = trials_df.select([fn.count(fn.when(fn.col(c).isNull(), c)).alias(c) for c in trials_df.columns])
null_columns = trials_df.select([count(when(col(c).isNull(), c)).alias(c) for c in trials_df.columns])
null_columns.show()


ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# Counting null values per column
# Only when that column is null
# nulls_df = trials_df.select([fn.count(fn.when(fn.col(c).isNull(), c)).alias(c) for c in columns])
# nulls_df.show()
# Throwing error

Data Cleaning
-flatten df
-address null values
-rename columns

In [ ]:
# checking the rows again in flattened format

new_trials_df = trials_df.select([*columns])

#### Clean Column names

In [ ]:
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [ ]:
new_trials_df.columns

['ageofparticipant',
 'branch',
 'name',
 'role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'conclusion',
 'sideeffectsonparticipant']

In [ ]:
# Store the columns in variable columns to remove the .annotations

columns = new_trials_df.columns
columns

['ageofparticipant',
 'branch',
 'name',
 'role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'conclusion',
 'sideeffectsonparticipant']

### Cleaning


In [ ]:
# flatten df
# address null values
# rename columns

In [ ]:
# Note that in the case study description the name of clinician is described as head_clincian's name
# and the role is called the assisting clinician

In [ ]:
# rename columns takes withColumnsRenamed and the dictionary of oldname:new name
new_column_names = {
    'ageofparticipant': 'age_of_participant'
    , 'branch': 'clinic_branch'
    , 'name': 'head_clinician'
    , 'role': 'assistants_role'
    , 'experimentenddate': 'experiment_end_date'
    , 'experimentstartdate': 'experiment_start_date'
    , 'noofhourspassedatfirstreaction': 'hours_passed_at_first_reaction'
    , 'sideeffectsonparticipant': 'observed_side_effect'
}
new_trials_df = new_trials_df.withColumnsRenamed(new_column_names)
new_trials_df.show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|
+------------------+-------------+--------------+---------------+---------+-----

In [ ]:
# Let's fix null values
new_trials_df.describe().show() 

In [ ]:
# missing value observed in 3 columns: assistants_role, hours_passed_at_first_reaction, conclusion

In [ ]:
# fill columns with null values
# Putting 0 for hours might indicate immediate side effect after drug use, so no need filling

new_trials_df = new_trials_df.na.fill({'conclusion': 'No entry', 'assistants_role': 'No entry'})

In [ ]:
new_trials_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistants_role: string (nullable = false)
 |-- drug_used: string (nullable = true)
 |-- experiment_end_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- hours_passed_at_first_reaction: long (nullable = true)
 |-- conclusion: string (nullable = false)
 |-- observed_side_effect: string (nullable = true)



In [ ]:
new_trials_df.select([count(when(col(a).isNull() | isnan(col(a)) | col(a).isin('', 'null', None), a)).alias(a) for a in new_trials_df.columns]).show()

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+----------+--------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|conclusion|observed_side_effect|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+----------+--------------------+
|                 0|            0|             0|              0|        0|                  0|                    0|                            73|         0|                   0|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+----------+--------------------+



#### Transformation steps
##### experiment start and end date to datetime

In [ ]:
# The date format used here is a unix time stamp...counts number of seconds from Jan 1, 1970 to now
new_trials_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistants_role: string (nullable = false)
 |-- drug_used: string (nullable = true)
 |-- experiment_end_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- hours_passed_at_first_reaction: long (nullable = true)
 |-- conclusion: string (nullable = false)
 |-- observed_side_effect: string (nullable = true)



In [ ]:
new_trials_df.describe().show()

23/09/07 19:27:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+--------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+--------------------+
|  count|              3586|         3586|          3586|           3586|     3586|                3586|                 3586|                          3513|         3586|                3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|         null|                null|
| stddev|2.3066401927555233|       

In [ ]:
import pyspark.sql.types as dtype
# or from pyspark.sql import types as dtype
import pyspark.sql.functions as fn
# importing the whole pyspark.sql function module

#### Date conversion from longtype to datetime

In [ ]:
# Convert data type from string to Longtype(integer)
# Divide by 1000
# Convert from unix to datetime

In [ ]:
## ----Coinversion steps

# With column to create new columns
# imported fn.col method on column of interest: experiment_end_date
# Apply cast data conversion
# pyspark.sql.types module as dtypes conversion applied on the column 
# /1000 cos we observed the data is more than 10 digits, probably in mili or micro secods, we need seconds
# Data is in unix and unix is in seconds
# So the whole datatype econversion is housed in our new column start_ts

new_trials_df.withColumn('start_ts', fn.col('experiment_end_date').cast(dtype.LongType())/1000).show(4)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-----------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|   start_ts|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-----------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|1.6198272E9|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|1.6198272E9|
|                17|

In [ ]:
## Now we Convert from unix to datetime, start date and end date
# I first wrap fn.col---to the end in a bracket

# new_trials_df.withColumn('start_ts', (fn.col('experiment_end_date').cast(dtype.LongType())/1000))
# Then i'll introduce the function.from unix timestamp method, part of the function module imported as fn

# new_trials_df.withColumn('start_ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtype.LongType())/1000))
# We introduce our date format

# new_trials_df.withColumn('start_ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtype.LongType())/1000),'yyyy-MM-dd HH:mm:ss.SSSS')
# In the documentation it says time stamp conversion returns a string, so further conversion will be needed

In [ ]:
new_trials_df.withColumn('start_ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtype.LongType())/1000,'yyyy-MM-dd HH:mm:ss.SSSS')).show(4)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+--------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|            start_ts|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+--------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|2021-05-01 03:00:...|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N

In [ ]:
# Lets check our column to see the data type, it will return a string 
new_trials_df.withColumn('start_ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtype.LongType())/1000,'yyyy-MM-dd HH:mm:ss.SSSS')).dtype

[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('head_clinician', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('observed_side_effect', 'string'),
 ('start_ts', 'string')]

In [ ]:
# lets convert from string to timestamp
# You can always seperate your columns by using a backslash and continue on the next line
# simply take the result of a line1, apply the backslash and use the withcolumn on the result of the first line as shown below
# Doesn't have to be indented just for readability

In [ ]:
newnew_trials_df =new_trials_df.withColumn('experiment_start_date', fn.from_unixtime(col('experiment_start_date').cast(dtype.LongType())/1000, 'yyyy-MM-dd HH:mm:ss.SSSS'))\
    .withColumn('experiment_start_date', col('experiment_start_date').cast(dtype.TimestampType()))\
        .withColumn('experiment_end_date', fn.from_unixtime(col('experiment_end_date').cast(dtype.LongType())/1000, 'yyyy-MM-dd HH:mm:ss.SSSS'))\
            .withColumn('experiment_end_date', col('experiment_end_date').cast(dtype.TimestampType()))

In [ ]:
# Unix to timestamp conversion # my code
# If you import the entire function module as fn use fn.col, fn.count
# If specific module is imported you can use col, count etc
df = new_trials_df.withColumn('start_date', fn.unix_timestamp(fn.col('experiment_start_date').cast(dtype.LongType())/1000,'yyyy-MM-dd HH:mm:ss.SSSS'))\
    .withColumn('start_date', fn.col('start_date').cast(dtype.TimestampType()))

In [ ]:
# partition by drug type for easy accessibility
# order by conclusion and sideeffect for easy readability/analysis
# Clinicians want to understand what they are reading


In [116]:
new_trials_df.show(4)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|2021-05-01 03:00:00|  2021-04-01 03:00:00|                            52|BP normalized|      rashes on neck|
|                14|      Ontario|          Saul|          nurse| Naproxen|2021-05-01 03:00:00|  2021-04-01 03:00:00|                            78|    Follow-up|                 N/A|
|                17|      Ontario|          Saul|          nurse|  Placebo|2021-

### ml engineers want the data to be saved in an easily retrievable format, so we do 
#### also want data partitioned by month

In [117]:
new_trials_df.columns

['age_of_participant',
 'clinic_branch',
 'head_clinician',
 'assistants_role',
 'drug_used',
 'experiment_end_date',
 'experiment_start_date',
 'hours_passed_at_first_reaction',
 'conclusion',
 'observed_side_effect']

In [125]:
# Reordered columns and pick out columns of interest for the clinicians
# Partition by so the clinicians can view the two trial drugs trials in different csv for easy accesibility
# Partition by is like using .loc to filter two different DFS and -to_csv into 2 csvs
new_trials_df = new_trials_df.select(['experiment_start_date', 'experiment_end_date', 'clinic_branch', 'drug_used', 'head_clinician', 'assistants_role', 'age_of_participant', 'hours_passed_at_first_reaction', 'conclusion', 'observed_side_effect'])

In [124]:
new_trials_df.show(2)

+---------------------+-------------------+-------------+---------+--------------+---------------+------------------+------------------------------+-------------+--------------------+
|experiment_start_date|experiment_end_date|clinic_branch|drug_used|head_clinician|assistants_role|age_of_participant|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+---------------------+-------------------+-------------+---------+--------------+---------------+------------------+------------------------------+-------------+--------------------+
|  2021-04-01 03:00:00|2021-05-01 03:00:00|      Ontario|  Placebo|          Saul|      therapist|                19|                            52|BP normalized|      rashes on neck|
|  2021-04-01 03:00:00|2021-05-01 03:00:00|      Ontario| Naproxen|          Saul|          nurse|                14|                            78|    Follow-up|                 N/A|
+---------------------+-------------------+-------------+---------+-------------

In [128]:
# sort data by experiment_start_date, header = True, partition into 2 csvs by drug used, mode of writing 
# is overwrite and folder where partitioned file saved is clinician folder
new_trials_df.sort('experiment_start_date').write.option('header', True)\
    .partitionBy(['drug_used'])\
    .mode('overwrite')\
    .csv('clinician')


### Derive Month from TimeStamp for ml engineers

In [129]:
# new column month and year added # and this is not saving to a csv
# parquet is a different file format unlike csvs, saves in column oriented rather than rows in csv
ml_trial_results = new_trials_df.withColumn('exp_start_month', fn.month(fn.col('experiment_start_date')))\
    .withColumn('exp_start_year', fn.year(col('experiment_start_date'))).sort('experiment_start_date')

In [131]:
ml_trial_results.show(4)

+---------------------+-------------------+-------------+---------+--------------+---------------+------------------+------------------------------+----------+--------------------+---------------+--------------+
|experiment_start_date|experiment_end_date|clinic_branch|drug_used|head_clinician|assistants_role|age_of_participant|hours_passed_at_first_reaction|conclusion|observed_side_effect|exp_start_month|exp_start_year|
+---------------------+-------------------+-------------+---------+--------------+---------------+------------------+------------------------------+----------+--------------------+---------------+--------------+
|  2021-02-01 02:00:00|2021-03-02 02:00:00|      Alberta| Naproxen|       Johnson|          nurse|                20|                            45| Follow-up|      rashes on neck|              2|          2021|
|  2021-02-01 02:00:00|2021-03-02 02:00:00|      Alberta| Naproxen|        Greene|      therapist|                16|                            75| No 

In [132]:
new_trials_df.withColumn('start_month', fn.month('experiment_start_date'))\
.sort('experiment_start_date').write.partitionBy('start_month')\
    .mode('overwrite').format('parquet').save('ml_engineers')

In [133]:
ml_trial_results.write.option('header', True)\
    .partitionBy(['exp_start_month', 'drug_used'])\
        .mode('overwrite')\
            .parquet('ml_dataset')